In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
# from MultiBert_BiLSTM_SelfAttention_FIX import NewsClassifier  # 假设你的模型定义在 model.py 文件中
# import MultiBert_BiLSTM_SelfAttention_FIX  # 假设你的模型定义在 model.py 文件中



/opt/conda/envs/default/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 加载BERT tokenizer和模型
# model_name = '../../../bert-base-multilingual-cased'
# tokenizer = BertTokenizer.from_pretrained(model_name)
# bert_model = BertModel.from_pretrained(model_name)

model_path = '../NewsAthm/bert-base-multilingual-cased'  ## 可更换

bert_model = BertModel.from_pretrained(model_path)

# 加载BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)


# 将模型移动到GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
print(device)
bert_model.to(device)


# 定义Self-Attention层
class SelfAttention(nn.Module):
    def __init__(self, hidden_size):
        super(SelfAttention, self).__init__()
        self.hidden_size = hidden_size
        self.projection = nn.Sequential(
            nn.Linear(hidden_size, 64),
            nn.ReLU(True),
            nn.Linear(64, 1)
        )

    def forward(self, encoder_outputs):
        energy = self.projection(encoder_outputs)
        weights = torch.softmax(energy.squeeze(-1), dim=1)
        outputs = (encoder_outputs * weights.unsqueeze(-1)).sum(dim=1)
        return outputs


cuda


In [4]:
# 定义模型
class NewsClassifier(nn.Module):
    # hidden_size = 128
    def __init__(self, num_classes, hidden_size, num_layers=2, bidirectional=True):
        super(NewsClassifier, self).__init__()
        # self.bert = BertModel.from_pretrained('../../bert-base-multilingual-cased')
        self.bert = bert_model # FIX

        # self.lstm = nn.LSTM(input_size=hidden_size, hidden_size=hidden_size, num_layers=num_layers, 
        #                     bidirectional=bidirectional, batch_first=True)
        self.lstm = nn.LSTM(input_size=bert_model.config.hidden_size, hidden_size=hidden_size, num_layers=num_layers, 
                            bidirectional=bidirectional, batch_first=True) # FIX
        
        self.attention = SelfAttention(hidden_size * (2 if bidirectional else 1))
        self.fc = nn.Linear(hidden_size * (2 if bidirectional else 1), num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        last_hidden_state = outputs.last_hidden_state
        lstm_outputs, _ = self.lstm(last_hidden_state)
        attention_outputs = self.attention(lstm_outputs)
        logits = self.fc(attention_outputs)
        return logits

In [5]:
# 设置超参数
# max_length = 256
max_length = 512
# origin:max_length = 512
# batch_size = 16
# epochs = 20
# learning_rate = 2e-5
# hidden_size = 768
hidden_size = 128
num_classes = 9

num_layers = 2
bidirectional = True

# 加载训练好的模型
modelNew_load_path = '../NewsAthmTask2Score/classificationModel/best_MultiBert_BiLSTM_SelfAttention_modelFIX_fold_5.pth'  ## 可更换
model = NewsClassifier(num_classes=num_classes, hidden_size=hidden_size, num_layers=num_layers, bidirectional=bidirectional)
model.load_state_dict(torch.load(modelNew_load_path))
model.to(device)
model.eval()

NewsClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [7]:
model_CLS_name = "mbert_BiLSTM_SelfAttention" ###!!!




# 定义类别列表
# categories = ['খেলাধুলা', 'রাজনীতি', 'বিনোদন', 'অর্থনীতি', 'আইন', 'শিক্ষা', 'বিজ্ঞান', 'লাইফস্টাইল', 'অন্যান্য']
# ['体育', '政治', '娱乐', '经济', '法律', '教育', '科学', '生活方式', '其他']
# categories = ['বিজ্ঞান', 'খেলাধুলা', 'লাইফস্টাইল', 'শিক্ষা', 'বিনোদন', 'অর্থনীতি', 'আইন', 'অন্যান্য', 'রাজনীতি'] # FIX
# ['科学'、'体育'、'生活方式'、'教育'、'娱乐'、'经济'、'法律'、'其他'、'政治']


# categories = ['বিজ্ঞান','রাজনীতি','বিনোদন','অন্যান্য','শিক্ষা','অর্থনীতি','খেলাধুলা','আইন','লাইফস্টাইল']  # FIX!!!!!!!!!!!!
categories = ['রাজনীত','লাইফস্টাইল','শিক্ষা','অর্থনীতি','খেলাধুলা','অন্যান্য','বিজ্ঞান','বিনোদন', 'আইন'] # FIX!!!!!!!!!!!!
# [‘政治’、‘生活方式’、‘教育’、‘经济’、‘体育’、‘其他’、‘科学’、‘娱乐’、‘法律’]


# ['科学'、'体育'、'生活方式'、'教育'、'娱乐'、'经济'、'法律'、'其他'、'政治']
# ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
# [‘科学’、‘政治’、‘娱乐’、‘其他’、‘教育’、‘经济’、‘体育’、‘法律’、‘生活方式’]
# ==============================================================================
# categories = ['খেলাধুলা', 'রাজনীতি', 'বিনোদন', 'অর্থনীতি', 'আইন', 'শিক্ষা', 'বিজ্ঞান', 'লাইফস্টাইল', 'অন্যান্য']
# ['体育', '政治', '娱乐', '经济', '法律', '教育', '科学', '生活方式', '其他']

# ['রাজনীত','লাইফস্টাইল','শিক্ষা','অর্থনীতি','খেলাধুলা','অন্যান্য','বিজ্ঞান','বিনোদন', 'আইন'].
# [‘政治’、‘生活方式’、‘教育’、‘经济’、‘体育’、‘其他’、‘科学’、‘娱乐’、‘法律’]
# ================================================================================
# 'খেলাধুলা' 体育 1 ---'রাজনীতি' 政治 8

# আইন 法律 6---> 'খেলাধুলা' 体育 1  预测为法律的应该为体育

# 'রাজনীতি' 政治 8 --->  'লাইফস্টাইল' 生活方式 2   更像是“健康”？

# 'অর্থনীতি' 经济 5--- 经济

# বিনোদন 娱乐 4 ---> শিক্ষা 教育 3 

# বিজ্ঞান 科学 0 --- 科学

# লাইফস্টাইল 生活方式 2 --- বিনোদন 娱乐 4 

# অন্যান্ 其他 7 --->  আইন 法律6 




# 0 1 2 3 4 5 6 7 8   3 教育 4娱乐
# ↓
# 0 8 4 7 3 5 1 6 2   3教育,7其他

# ['বিজ্ঞান','রাজনীতি','বিনোদন','অন্যান্য','শিক্ষা','অর্থনীতি','খেলাধুলা','আইন','লাইফস্টাইল']
# 定义数据处理函数
def preprocess_data(text, tokenizer, max_length):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_length,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True
    )
    return encoding['input_ids'].to(device), encoding['attention_mask'].to(device)

# 定义预测函数
def predict(model, input_ids, attention_mask):
    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        _, preds = torch.max(outputs, dim=1)
    return preds.item(), categories[preds.item()]
    # return preds.item()




In [9]:
# 加载孟加拉语新闻语料
news_data_path = './datasets/news_20240302_20240318.csv'
news_data = pd.read_csv(news_data_path)

news_data['pub_time'] = pd.to_datetime(news_data['pub_time'])


# 对每天的新闻数据进行处理
start_date = pd.to_datetime('2024-03-02')
end_date = pd.to_datetime('2024-03-18')
current_date = start_date


while current_date <= end_date:
    print(current_date)
    date_str = current_date.strftime('%Y-%m-%d')
    print(date_str)

    daily_news_data = news_data[news_data['pub_time'] == date_str]
    

    daily_news_data = daily_news_data.dropna(subset=['category1','body'])

    
    # 对不属于预定义类别的新闻进行类别预测
    predicted_categories = []
    predicted_categories_id = []

    cnt = 0
    for idx, row in daily_news_data.iterrows():
        cnt+=1
        if cnt%200 == 0: 
            print(cnt)
        if row['category1'] not in categories:
            input_ids, attention_mask = preprocess_data(row['body'], tokenizer, max_length)
            pred_id, predicted_category = predict(model, input_ids, attention_mask)
            predicted_categories.append(predicted_category)
            predicted_categories_id.append(pred_id)
        else:
            predicted_categories.append(row['category1'])
            predicted_categories_id.append(row['category1'])

    
    # print(predicted_categories)
    # 将预测后的类别替换原有的category1列
    daily_news_data['category1'] = predicted_categories
    print(predicted_categories_id)
    
    # 将处理后的数据存储到CSV文件中
    output_file = f'./datasets/{model_CLS_name}/news_{date_str}_processed_{model_CLS_name}.csv'
    daily_news_data.to_csv(output_file, index=False)
    
    # print(f"Processing date: {date_str}")
    # print(f"Number of news articles: {len(daily_news_data)}")
    # print(f"Columns in daily_news_data: {daily_news_data.columns}")
    # print(f"Output file: {output_file}")
    current_date += pd.Timedelta(days=1)

2024-03-02 00:00:00
2024-03-02
200
[3, 0, 0, 0, 0, 1, 1, 0, 4, 0, 0, 3, 0, 0, 8, 8, 1, 2, 0, 3, 0, 0, 3, 0, 7, 8, 0, 1, 0, 3, 1, 3, 3, 8, 6, 0, 0, 3, 5, 3, 8, 3, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 0, 0, 1, 0, 3, 8, 3, 2, 7, 8, 8, 0, 3, 3, 8, 2, 3, 8, 7, 8, 3, 8, 3, 5, 3, 3, 2, 8, 6, 3, 3, 2, 3, 8, 2, 2, 0, 0, 3, 2, 6, 0, 8, 8, 1, 3, 8, 4, 7, 7, 7, 4, 7, 7, 4, 7, 7, 4, 3, 3, 3, 0, 3, 3, 7, 5, 3, 7, 3, 8, 3, 3, 8, 2, 2, 3, 0, 2, 4, 8, 3, 6, 0, 1, 0, 4, 2, 1, 3, 2, 8, 8, 7, 7, 7, 7, 5, 0, 6, 0, 8, 6, 0, 5, 5, 5, 5, 2, 2, 5, 6, 5, 7, 5, 5, 6, 5, 6, 3, 3, 1, 2, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 3, 8, 0, 4, 8, 2, 6, 3, 3, 8, 8, 0, 0, 5, 3, 3, 3, 0, 7, 0, 3, 2, 7, 1, 2, 3, 2, 8, 3, 3, 2, 6, 4, 8, 0, 6, 0, 2, 5, 3, 6, 0, 0, 3, 3, 3, 0, 2, 3, 0, 0, 0, 1, 8, 2, 0, 2, 7, 2, 0, 0, 5, 3, 0, 3, 8, 3, 8, 3, 0, 5, 5, 'বিনোদন', 5, 3, 1, 'লাইফস্টাইল', 6, 0, 3, 2, 0, 0, 0, 4, 8, 5, 6, 3, 5, 5, 4, 3, 3, 3, 3, 3, 3, 4, 8, 1, 1, 5, 6, 0, 5, 0, 3, 3, 3, 8, 2, 0, 3, 8, 1, 4, 3, 0, 3, 3, 0, 8, 2, 1

In [ ]:
------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
[3, 0, 0, 0, 0, 1, 1, 0, 4, 0, 0, 3, 0, 0, 8, 8, 1, 2, 0, 3, 0, 0, 3, 0, 7, 8, 0, 1, 0, 3, 1, 3, 3, 8, 6, 0, 0, 3, 5, 3, 8, 3, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 0, 0, 1, 0, 3, 8, 3, 2, 7, 8, 8, 0, 3, 3, 8, 2, 3, 8, 7, 8, 3, 8, 3, 5, 3, 3, 2, 8, 6, 3, 3, 2, 3, 8, 2, 2, 0, 0, 3, 2, 6, 0, 8, 8, 1, 3, 8, 4, 7, 7, 7, 4, 7, 7, 4, 7, 7, 4, 3, 3, 3, 0, 3, 3, 7, 5, 3, 7, 3, 8, 3, 3, 8, 2, 2, 3, 0, 2, 4, 8, 3, 6, 0, 1, 0, 4, 2, 1, 3, 2, 8, 8, 7, 7, 7, 7, 5, 0, 6, 0, 8, 6, 0, 5, 5, 5, 5, 2, 2, 5, 6, 5, 7, 5, 5, 6, 5, 6, 3, 3, 1, 2, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 3, 8, 0, 4, 8, 2, 6, 3, 3, 8, 8, 0, 0, 5, 3, 3, 3, 0, 7, 0, 3, 2, 7, 1, 2, 3, 2, 8, 3, 3, 2, 6, 4, 8, 0, 6, 0, 2, 5, 3, 6, 0, 0, 3, 3, 3, 0, 2, 3, 0, 0, 0, 1, 8, 2, 0, 2, 7, 2, 0, 0, 5, 3, 0, 3, 8, 3, 8, 3, 0, 5, 5, 'বিনোদন', 5, 3, 1, 'লাইফস্টাইল', 6, 0, 3, 2, 0, 0, 0, 4, 8, 5, 6, 3, 5, 5, 4, 3, 3, 3, 3, 3, 3, 4, 8, 1, 1, 5, 6, 0, 5, 0, 3, 3, 3, 8, 2, 0, 3, 8, 1, 4, 3, 0, 3, 3, 0, 8, 2, 1, 3, 1, 4, 8, 0, 3, 3, 3, 3, 3, 'অন্যান্য', 'অন্যান্য', 8, 8, 8, 3, 3, 4, 3, 8, 2, 0, 8, 2, 2, 0, 3, 3, 8, 3, 0, 8, 2, 3, 5, 3, 8, 2, 6, 3, 0, 3, 0, 3, 4, 0, 8, 3, 7, 3, 0, 4, 3, 5, 3, 4, 0, 0, 'অন্যান্য', 8, 0, 0, 6, 6, 0, 2, 3, 3, 2, 8, 0, 4, 0, 3, 8, 4, 8, 5]

In [ ]:
# 对不属于预定义类别的新闻进行类别预测
    predicted_categories = []
    cnt = 0
    for idx, row in daily_news_data.iterrows():
        cnt+=1
        if cnt%100 == 0: 
            print(cnt)
        if row['category1'] not in categories:
            input_ids, attention_mask = preprocess_data(row['body'], tokenizer, max_length)
            predicted_category = predict(model, input_ids, attention_mask)
            predicted_categories.append(predicted_category)
        else:
            predicted_categories.append(row['category1'])

In [43]:
# 加载数据
# file_path = '../../../datasets_FIX2/FIX2_deduplicated_mangoNews_Nums3000p_CategoryMerge_new_undersampled_Example.csv'
file_path = '../NewsAthm/datasets_FIX2/FIX2_deduplicated_mangoNews_Nums3000p_CategoryMerge_new_undersampled.csv'

data_UUUUU = pd.read_csv(file_path,low_memory=False,lineterminator="\n")

texts = data_UUUUU['body'].tolist()
labels = data_UUUUU['category1'].tolist()



In [44]:
data_UUUUU.head()

,id,website_id,request_url,response_url,category1,category2,title,abstract,body,pub_time,cole_time,images,language_id,md5\r
0,17333224,2281,https://www.jugantor.com/sports/640555/%E0%A6%...,https://www.jugantor.com/sports/640555/%E0%A6%...,খেলাধুলা,NaN,বিশ্বকাপে কোচের যে সিদ্ধান্তে বিস্মিত হন ডি মা...,কাতার বিশ্বকাপের ফাইনালে ফ্রান্সকে নাকাল করে ছ...,কাতার বিশ্বকাপের ফাইনালে ফ্রান্সকে নাকাল করে ছ...,1/2/2023 00:00:00,22/9/2023 10:10:18,"[""https://www.jugantor.com/assets/news_photos/...",1779,15c4e2f5f9cf435a7f1b5256b4b358ab\r
1,17150916,2270,https://dmpnews.org/%e0%a6%9f%e0%a6%bf%e0%a6%a...,https://dmpnews.org/%e0%a6%9f%e0%a6%bf%e0%a6%a...,খেলাধুলা,NaN,টিভিতে আজকের খেলা,ডিএমপি নিউজঃ খেলাধুলা মানুষের জীবনে বিনোদনের এ...,ডিএমপি নিউজঃ খেলাধুলা মানুষের জীবনে বিনোদনের এ...,20/9/2018 16:53:29,22/9/2023 01:28:16,"[""https://dmpnews.org/wp-content/uploads/2018/...",1779,4331bda08631db3cf58e36ed2d18b30c\r
2,17092926,2270,https://dmpnews.org/%e0%a6%aa%e0%a7%8d%e0%a6%b...,https://dmpnews.org/%e0%a6%aa%e0%a7%8d%e0%a6%b...,খেলাধুলা,NaN,প্রত্যাশিত বড় জয় নিয়েই বিশ্বকাপের মূল পর্বে বা...,সাকিব আল হাসানের অনবদ্য অলরাউন্ডার পারফমেন্সে ...,সাকিব আল হাসানের অনবদ্য অলরাউন্ডার পারফমেন্সে ...,21/10/2021 20:39:58,21/9/2023 22:52:51,"[""https://dmpnews.org/wp-content/uploads/2021/...",1779,024170153e9e7a6c9d78a877cb431650\r
3,17152100,2270,https://dmpnews.org/%e0%a6%ac%e0%a6%be%e0%a6%b...,https://dmpnews.org/%e0%a6%ac%e0%a6%be%e0%a6%b...,খেলাধুলা,NaN,বার্সেলোনায় ফিরতে আকুল নেইমার,নিজের পুরনো ক্লাব বার্সেলোনায় ফিরতে আকুল ব্রাজ...,নিজের পুরনো ক্লাব বার্সেলোনায় ফিরতে আকুল ব্রাজ...,13/1/2019 10:12:00,22/9/2023 01:31:27,"[""https://dmpnews.org/wp-content/uploads/2019/...",1779,e8e211b5ff7d2b5556d8675831c87bf0\r
4,18749934,2277,https://www.dhakatimes24.com/2021/02/06/201644,https://www.dhakatimes24.com/2021/02/06/201644,খেলাধুলা,NaN,দিনের প্রথম উইকেট মুশফিক,অধিনায়ক মুমিনুল হক এবং উইকেটকিপার ব্যাটসম্যান ...,অধিনায়ক মুমিনুল হক এবং উইকেটকিপার ব্যাটসম্যান ...,6/2/2021 10:49:00,30/9/2023 13:37:31,"[""https://www.dhakatimes24.com/assets/news_pho...",1779,4e18e2c6d69d7dc1a2d744f882b4e74e\r


In [45]:
# 对标签进行编码
unique_labels = list(set(labels))
print(unique_labels)
label_to_id = {label: i for i, label in enumerate(unique_labels)}
print(label_to_id)
labels = [label_to_id[label] for label in labels]
# print(labels)
num_classes = len(unique_labels)
print(num_classes)

['বিজ্ঞান', 'খেলাধুলা', 'লাইফস্টাইল', 'শিক্ষা', 'বিনোদন', 'অর্থনীতি', 'আইন', 'অন্যান্য', 'রাজনীতি']
{'বিজ্ঞান': 0, 'খেলাধুলা': 1, 'লাইফস্টাইল': 2, 'শিক্ষা': 3, 'বিনোদন': 4, 'অর্থনীতি': 5, 'আইন': 6, 'অন্যান্য': 7, 'রাজনীতি': 8}
9


In [46]:
# 加载数据
# file_path = '../../../datasets_FIX2/FIX2_deduplicated_mangoNews_Nums3000p_CategoryMerge_new_undersampled_Example.csv'
file_path = './datasets/news_2024-03-02_processed_mbert_mbert_BiLSTM_SelfAttention.csv'

data_U = pd.read_csv(file_path,low_memory=False,lineterminator="\n")

data_U.head()

,id,website_id,request_url,response_url,category1,category2,title,abstract,body,pub_time,cole_time,images,language_id,md5
0,20044932,2268,https://mzamin.com/news.php?news=100055,https://mzamin.com/news.php?news=100055,শিক্ষা,NaN,‘প্রত্যাগত অভিবাসী কর্মীদের প্রশিক্ষণের বিকল্প...,প্রত্যাগত অভিবাসী কর্মীদের পুন:একত্রীকরণ তথা স...,প্রত্যাগত অভিবাসী কর্মীদের পুন:একত্রীকরণ তথা স...,2024-03-02,2024-03-12 08:00:34,"[""https://mzamin.com/uploads/news/main/100055_...",1779,aff97b7a708c043c23f3b769bceacfcb
1,20044933,2268,https://mzamin.com/news.php?news=100034,https://mzamin.com/news.php?news=100034,বিজ্ঞান,NaN,নতুন রাজনৈতিক দলের ঘোষণা দিলেন শাহেদা ওবায়েদ,দেশের নতুন রাজনৈতিক দল গঠনের ঘোষণা দিয়েছেন বিএ...,দেশের নতুন রাজনৈতিক দল গঠনের ঘোষণা দিয়েছেন বিএ...,2024-03-02,2024-03-12 08:00:35,"[""https://mzamin.com/uploads/news/main/100034_...",1779,32b9ec59f6855bdb48e93e9930915236
2,20044934,2268,https://mzamin.com/news.php?news=100033,https://mzamin.com/news.php?news=100033,বিজ্ঞান,NaN,বেইলি রোডে আগুনে প্রাণহানির ঘটনায় মোদির শোক,প্রধানমন্ত্রী শেখ হাসিনাকে লেখা চিঠিতে রাজধানী...,প্রধানমন্ত্রী শেখ হাসিনাকে লেখা চিঠিতে রাজধানী...,2024-03-02,2024-03-12 08:00:35,"[""https://mzamin.com/uploads/news/main/8874_mo...",1779,677a0604c17121f0ae35a44e0370e25e
3,20044935,2268,https://mzamin.com/news.php?news=100032,https://mzamin.com/news.php?news=100032,বিজ্ঞান,NaN,"সরকার জুলুম করতে পারে, জিনিসের দাম কমাতে পারে ...","সরকার জুলুম করতে পারে, জিনিসের দাম কমাতে পারে ...","সরকার জুলুম করতে পারে, জিনিসের দাম কমাতে পারে ...",2024-03-02,2024-03-12 08:00:36,"[""https://mzamin.com/uploads/news/main/100032_...",1779,323e54d5efed89e92c0e14fa514a9d37
4,20044936,2268,https://mzamin.com/news.php?news=100029,https://mzamin.com/news.php?news=100029,বিজ্ঞান,NaN,ফখরুলের সঙ্গে গণতন্ত্র মঞ্চ ও ১২ দলের বৈঠক,বিএনপি মহাসচিব মির্জা ফখরুল ইসলাম আলমগীরের সঙ্...,বিএনপি মহাসচিব মির্জা ফখরুল ইসলাম আলমগীরের সঙ্...,2024-03-02,2024-03-12 08:00:37,"[""https://mzamin.com/uploads/news/main/92116_B...",1779,df9d7a02ca1f3fe174e95a368f1b606d
